<a href="https://colab.research.google.com/github/Tatsuro0726/Scraping/blob/master/Chapter7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Chapter7-1: 類似画像の検出を使用
#### 簡単な形状認識 - Average Hash
- 画像を比較可能なハッシュ値で表す。ハッシュ関数:MD5, SHA256などを使う。⇒ データの値を要約したハッシュ値を得られる。
- 画像データは一般的に同一ではない(サイズとか色調, jpeg/pngなど)のため、ハッシュ関数を使えない。
- 類似度計算に使用するAverage Hashが役立つ
  1. 画像のサイズを8×8
  1. 色をグレースケールに変換
  1. 画像の各ピクセルの平均値を計算
  1. 各ピクセルの濃淡が平均より大きければ1, 平均以下なら0とする
- 上記の手順で、8×8=64bitのハッシュ値が得られる。⇒ 64ビットを比較するだけで済む。

In [ ]:
# Pythonの画像ライブラリーPillowをインストール
!pip install Pillow

In [ ]:
# avghash
from PIL import Image
import numpy as np

# 画像データをAverage Hashに変換
def average_hash(fname, size = 16):
  img = Image.open(fname)
  img = img.convert('L') # グレースケールに変換
  img = img.resize((size, size), Image.ANTIALIAS) # リサイズ　アンチエイリアス：境界線を滑らかに見せる
  pixel_data = img.getdata() # ピクセルデータを得る
  pixels = np.array(pixel_data) # numpy配列に変換
  pixels = pixels.reshape((size, size)) # 2次元配列に変換
  avg = pixels.mean() # 算術平均を計算
  diff = 1 * (pixels > avg) # 平均より大きければ1, 平均以下で0に変換
  return diff

# 2進数とみなしてハッシュ値に変換
def np2hash(n):
  bhash = []
  for nl in ahash.tolist():
    sl = [str(i) for i in nl]
    s2 = ''.join(sl)
    i = int(s2,2) # 2進数を整数に
    bhash.append('%04x' % i)
  return ''.join(bhash)

# Average Hashを表示
ahash = average_hash('./skytree.jpg')
print(ahash)
print(np2hash(ahash))

[[0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0]
 [0 0 1 1 1 1 0 1 1 1 0 1 0 0 0 0]
 [1 1 1 1 1 0 0 1 1 1 1 1 0 1 1 1]
 [1 1 1 1 1 0 0 0 1 1 1 1 0 0 1 1]
 [1 1 1 1 1 0 0 0 1 1 1 1 0 0 1 1]
 [1 1 1 1 1 0 0 0 1 1 1 1 0 0 1 1]
 [1 1 1 1 1 0 0 0 1 1 1 1 1 0 1 1]
 [1 1 1 1 0 0 0 0 1 1 1 1 1 1 1 1]
 [1 1 1 1 0 0 0 0 1 1 1 1 0 1 1 1]
 [1 1 1 1 0 0 0 0 1 1 1 1 0 1 1 1]
 [1 1 1 0 0 0 0 0 1 1 1 1 1 0 1 1]
 [1 1 1 0 0 0 0 0 0 1 1 1 1 0 1 1]
 [1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 1]
 [1 1 0 0 0 0 0 0 0 1 1 1 1 1 1 1]
 [1 1 0 0 0 0 0 0 0 1 1 1 0 0 1 1]
 [1 0 0 0 0 0 0 0 0 1 1 1 1 0 1 1]]
1f803dd0f9f7f8f3f8f3f8f3f8fbf0fff0f7f0f7e0fbe07be07fc07fc073807b


In [ ]:
from PIL import Image
import numpy as np
import os, re

# ファイルパスの指定
search_dir = './drive/My Drive/Colab Folder/images/flowers'
cache_dir = './drive/My Drive/Colab Folder/images/cache_avhash'

# Cacheフォルダ作成
if not os.path.exists(cache_dir):
  os.mkdir(cache_dir)

# 画像データをAverage hashに変換
def average_hash(fname, size=16):
  fname2 = fname[len(search_dir):]
  # 画像をキャッシュしておく
  cache_file = cache_dir + '/' + fname2.replace('/', '_') + 'csv'
  if not os.path.exists(cache_file): # ハッシュを作成
    img = Image.open(fname)
    img = img.convert('L').resize((size, size), Image.ANTIALIAS)
    pixels = np.array(img.getdata()).reshape((size, size))
    avg = pixels.mean()
    px = 1 * (pixels > avg)
    np.savetxt(cache_file, px, fmt='%.0f', delimiter=',')
  else: # すでにキャッシュがあればファイルから読み込み
    px = np.loadtxt(cache_file, delimiter=',')
  return px

# 簡単にハミング距離を求める
def hamming_dist(a, b):
  aa = a.reshape(1,-1) # 1次元配列に変換
  ab = b.reshape(1,-1)
  dist = (aa != ab).sum()
  return dist

# すべてのディレクトリを列挙
def enum_all_files(path):
  for root, dirs, files in os.walk(path):
    for f in files:
      fname = os.path.join(root, f)
      if re.search(r'\.(jpg|jpeg|png)$',fname):
        yield fname

# 画像を検索
def find_image(fname, rate):
  src = average_hash(fname)
  for fname in enum_all_files(search_dir):
    dst = average_hash(fname)
    diff_r = hamming_dist(src, dst) / 256
    # print('[check]', fname)
    if diff_r < rate:
      yield (diff_r, fname)

# 検索
srcfile = search_dir + '/rose/mountain_rose_s_000071.png'
html = ''
sim = list(find_image(srcfile, 0.25))
sim = sorted(sim, key=lambda x: x[0])
for r, f in sim:
  print(r, '>', f)
  s = '<div style="float:left;"><h3>[差異:' + str(r) + '-' + \
    os.path.basename(f) + ']</h3>'+ \
    '<p><a href="' + f + '"><img src="' + f + '" width=400>'+ \
    '</a></p></div>'
  html += s

# html出力
html = """<html><body><h3>元画像</h3><p>
<img src='{0} width=400></p>{1}
</body></html>""".format(srcfile, html)
with open("./avhash-search-output.html", "w", encoding="utf-8") as f:
  f.write(html)
print('ok')

0.0 > ./drive/My Drive/Colab Folder/images/flowers/rose/mountain_rose_s_000071.png
0.2421875 > ./drive/My Drive/Colab Folder/images/flowers/tulip/tulip_s_000227.png
ok


### Chapter7-2: CNNで画像分類に挑戦しよう
CNNでカラー画像の分類に挑戦する

In [11]:
# 画像データをPythonのデータ型に変換する
from sklearn.model_selection import train_test_split
from PIL import Image
import os, glob
import numpy as np

# 分類対象のカテゴリーを選ぶ
target_dir = './drive/My Drive/Colab Folder/images/household_furniture' 
categories = ['bed','chair','couch','table','wardrobe']
nb_classes = len(categories)

# 画像サイズの指定
image_w = 64
image_h = 64
pixles = image_w * image_h * 3

# 画像データの読み込み
X = []
Y = []
for idx, cat in enumerate(categories): # idx:繰り返しのインデックス, categoriesリストの要素
  # ラベル指定
  label = [0 for i in range(nb_classes)]
  label[idx] = 1
  # 画像
  image_dir = target_dir +'/'+ cat
  file = glob.glob(image_dir + '/*.png')
  for i, f in enumerate(file):
    img = Image.open(f)
    img = img.convert("RGB")
    img = img.resize((image_w,image_h))
    data = np.asarray(img)
    X.append(data)
    Y.append(label)
    if i % 10 == 0:
      print(i, '\n', data)
X = np.array(X)
Y = np.array(Y)

# 学習データとテストデータを分ける
X_train, X_test, y_train, y_test = \
train_test_split(X,Y)
xy = (X_train,X_test,y_train,y_test)
np.save('./drive/My Drive/Colab Folder/images/5obj.npy', xy)

print('oke,',len(Y))

0 
 [[[ 69  30  33]
  [ 70  31  34]
  [ 72  33  36]
  ...
  [234 242 240]
  [221 233 231]
  [214 228 227]]

 [[ 69  30  33]
  [ 70  31  34]
  [ 72  33  37]
  ...
  [232 242 240]
  [218 233 231]
  [211 228 226]]

 [[ 70  32  35]
  [ 71  33  36]
  [ 74  34  38]
  ...
  [229 243 239]
  [214 233 230]
  [206 228 225]]

 ...

 [[ 95  60  55]
  [ 97  62  57]
  [ 99  64  59]
  ...
  [155 133 131]
  [154 132 130]
  [153 132 130]]

 [[ 94  59  56]
  [ 95  60  58]
  [ 98  63  61]
  ...
  [152 131 130]
  [150 129 129]
  [149 128 128]]

 [[ 93  58  56]
  [ 95  60  58]
  [ 98  63  62]
  ...
  [151 130 130]
  [148 128 128]
  [147 126 127]]]
10 
 [[[254 244 215]
  [252 242 219]
  [248 239 224]
  ...
  [210 200 185]
  [205 196 179]
  [203 195 176]]

 [[254 247 224]
  [252 245 226]
  [248 242 231]
  ...
  [213 208 197]
  [208 204 191]
  [206 202 188]]

 [[254 253 241]
  [252 252 243]
  [247 250 246]
  ...
  [220 226 223]
  [215 221 216]
  [212 218 214]]

 ...

 [[101  91  85]
  [ 98  90  87]
  [ 91  88 

In [15]:
# CNNを実践してみよう
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPool2D
from keras.layers import  Activation, Dropout, Flatten, Dense
import numpy as np

# カテゴリーの指定
categories = ['bed','chair','couch','table','wardrobe']
nb_classes = len(categories)
# 画像サイズのして
image_w = 64
image_h = 64

# データをロード
X_train, X_test, y_train, y_test = np.load('./drive/My Drive/Colab Folder/images/5obj.npy',allow_pickle=True)
# データを正規化する
X_train = X_train.astype('float') / 256
X_test = X_test.astype('float') / 256
print('X_train shape:', X_train.shape)

# モデル構築
in_shape = X_train.shape[1:]
model = Sequential()
model.add(Conv2D(32, 3, input_shape=in_shape))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3,3)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3,3)))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# モデルを訓練する
model.fit(X_train, y_train, batch_size=32, epochs=50)

# モデルを評価する
score = model.evaluate(X_test, y_test)
print('loss=', score[0])
print('accuracy=', score[1])


X_train shape: (375, 64, 64, 3)
Epoch 1/50
12/12 [==============================] - 4s 318ms/step - loss: 1.0315 - accuracy: 0.1973
Epoch 2/50
12/12 [==============================] - 4s 314ms/step - loss: 0.4775 - accuracy: 0.3307
Epoch 3/50
12/12 [==============================] - 4s 316ms/step - loss: 0.4760 - accuracy: 0.3413
Epoch 4/50
12/12 [==============================] - 4s 313ms/step - loss: 0.4643 - accuracy: 0.3787
Epoch 5/50
12/12 [==============================] - 4s 316ms/step - loss: 0.4578 - accuracy: 0.4160
Epoch 6/50
12/12 [==============================] - 4s 316ms/step - loss: 0.4290 - accuracy: 0.4720
Epoch 7/50
12/12 [==============================] - 4s 315ms/step - loss: 0.4071 - accuracy: 0.5013
Epoch 8/50
12/12 [==============================] - 4s 313ms/step - loss: 0.4118 - accuracy: 0.4907
Epoch 9/50
12/12 [==============================] - 4s 316ms/step - loss: 0.3699 - accuracy: 0.5573
Epoch 10/50
12/12 [==============================] - 4s 316ms/step -

[0.9590681195259094, 0.5440000295639038]